# Data Exploration

In [1]:
import matplotlib.pyplot as plt
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from blockchain.lib.data.historical import *

In [2]:
api_key = "f9698sEkpQ_kUyz7ThBs"
qg = QuandlGetter(api_key)
df = qg.get(market=MARKET_KRAKEN)
df.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price,price,market,crypto_name,base
Date,,,,,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522,841.835522,KRAKEN,BTC,USD
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269,839.156269,KRAKEN,BTC,USD
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913,831.572913,KRAKEN,BTC,USD
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794,844.938794,KRAKEN,BTC,USD
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709,890.671709,KRAKEN,BTC,USD


In [3]:
sct=go.Scatter(x=df.index, y=df["price"])
sct2=go.Scatter(x=df.index, y=df["Close"])
py.iplot([sct, sct2])

In [4]:
pg = PoliniexGetter(period=60*60*24, start_date="2016-07-01")
btc_eth = pg.get(crypto_name=CRYPTO_BTC_ETH, market="poliniex")
btc_eth["USDWeightedAverage"] = btc_eth["weightedAverage"]*df["Weighted Price"]
btc = pg.get(crypto_name="USDT_BTC", market="poliniex")
eth = pg.get(crypto_name="USDT_ETH", market="poliniex")

In [5]:
sct_eth=go.Scatter(x=btc_eth.index, y=btc_eth["USDWeightedAverage"])
sct=go.Scatter(x=btc.index, y=btc["weightedAverage"]/10)
sct_eth2=go.Scatter(x=eth.index, y=eth["weightedAverage"])
py.iplot([sct_eth, sct, sct_eth2])

In [6]:
btc["weightedAverage"].corr(eth["weightedAverage"].shift(1))

0.94154424859245345

In [7]:
btc["weightedAverage"].corr(eth["weightedAverage"].shift(1))

0.94154424859245345

In [8]:
rcorr=pd.rolling_corr(btc["weightedAverage"],eth["weightedAverage"], 10)
sct_corr=go.Scatter(x=rcorr.index, y=rcorr)
sct=go.Scatter(x=btc.index, y=btc["weightedAverage"]/1000)
sct_eth2=go.Scatter(x=eth.index, y=eth["weightedAverage"]/100)
py.iplot([sct_corr, sct, sct_eth2])

/media/niki/DATA/.pyenv/versions/3.5.2/envs/blockchain/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning:

pd.rolling_corr is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=10).corr(other=<Series>)



In [9]:
from blockchain.lib.data.dataset.dataset import Dataset
x, y = Dataset.create_series_dataset(btc, eth, ["price"], "price", shuffle=False)

/media/niki/DATA/private/blockchainanalysis/blockchain/lib/data/dataset/dataset.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [11]:
from blockchain.lib.model.lstm_model import LSTMModel

In [14]:
config = {
    "log_dir": './ops_logs/lstm',
    "len_sequence": 10,
    "rnn_layers": [{'num_units': 5}],
    "dense_layers": None,
    "training_steps": 10000,
    "print_step": 10000 / 10,
    "batch_size": 100,
    "ratio_train_val_test": [0.1,0.2],
    "column": ["price"],
    "label": "price",
    "early_stopping": 10000,
    "num_unit": 10,
    "learning_rate": 1e-4,
    "optimizer": "Adam"
}


lstm = LSTMModel(config)

In [15]:
lstm.fit(btc, eth)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_random_seed': None, '_num_worker_replicas': 0, '_save_checkpoints_steps': None, '_environment': 'local', '_task_type': None, '_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './ops_logs/lstm', '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f87b1a39438>, '_keep_checkpoint_max': 5, '_is_chief': True, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_session_config': None, '_save_checkpoints_secs': 600, '_evaluation_master': ''}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Use `tf.contrib.losses.mean_squared_error` and explicit logits computation.
Instructions for updating:
Use tf.losses.mean_squared_error instead.


/media/niki/DATA/private/blockchainanalysis/blockchain/lib/data/dataset/dataset.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./ops_logs/lstm/model.ckpt.
INFO:tensorflow:step = 1, loss = 792.38
INFO:tensorflow:global_step/sec: 211.283
INFO:tensorflow:step = 101, loss = 753.705 (0.474 sec)
INFO:tensorflow:global_step/sec: 172.516
INFO:tensorflow:step = 201, loss = 833.775 (0.580 sec)
INFO:tensorflow:global_step/sec: 179.834
INFO:tensorflow:step = 301, loss = 758.071 (0.556 sec)
INFO:tensorflow:global_step/sec: 222.541
INFO:tensorflow:step = 401, loss = 600.099 (0.449 sec)
INFO:tensorflow:global_step/sec: 194.366
INFO:tensorflow:step = 501, loss = 1026.45 (0.514 sec)
INFO:tensorflow:global_step/sec: 220.889
INFO:tensorflow:step = 601, loss = 822.835 (0.453 sec)
INFO:tensorflow:global_step/sec: 218.592
INFO:tensorflow:step = 701, loss = 839.31 (0.457 sec)
INFO:tensorflow:global_step/sec: 214.214

INFO:tensorflow:global_step/sec: 218.697
INFO:tensorflow:step = 4501, loss = 823.681 (0.457 sec)
INFO:tensorflow:global_step/sec: 218.193
INFO:tensorflow:step = 4601, loss = 669.932 (0.458 sec)
INFO:tensorflow:global_step/sec: 228.509
INFO:tensorflow:step = 4701, loss = 651.091 (0.438 sec)
INFO:tensorflow:global_step/sec: 210.366
INFO:tensorflow:step = 4801, loss = 785.546 (0.475 sec)
INFO:tensorflow:global_step/sec: 218.703
INFO:tensorflow:step = 4901, loss = 673.59 (0.457 sec)
INFO:tensorflow:global_step/sec: 218.578
INFO:tensorflow:step = 5001, loss = 651.026 (0.457 sec)
INFO:tensorflow:global_step/sec: 179.511
INFO:tensorflow:step = 5101, loss = 823.913 (0.557 sec)
INFO:tensorflow:global_step/sec: 180.528
INFO:tensorflow:step = 5201, loss = 824.824 (0.554 sec)
INFO:tensorflow:global_step/sec: 219.022
INFO:tensorflow:step = 5301, loss = 960.135 (0.457 sec)
INFO:tensorflow:global_step/sec: 225.946
INFO:tensorflow:step = 5401, loss = 767.474 (0.443 sec)
INFO:tensorflow:global_step/sec

INFO:tensorflow:step = 12901, loss = 614.828 (0.615 sec)
INFO:tensorflow:global_step/sec: 168.206
INFO:tensorflow:step = 13001, loss = 725.603 (0.595 sec)
INFO:tensorflow:global_step/sec: 120.46
INFO:tensorflow:step = 13101, loss = 902.42 (0.830 sec)
INFO:tensorflow:global_step/sec: 154.324
INFO:tensorflow:step = 13201, loss = 653.758 (0.648 sec)
INFO:tensorflow:global_step/sec: 179.484
INFO:tensorflow:step = 13301, loss = 616.467 (0.557 sec)
INFO:tensorflow:global_step/sec: 137.016
INFO:tensorflow:step = 13401, loss = 607.589 (0.730 sec)
INFO:tensorflow:global_step/sec: 158.549
INFO:tensorflow:step = 13501, loss = 588.166 (0.631 sec)
INFO:tensorflow:global_step/sec: 145.835
INFO:tensorflow:step = 13601, loss = 585.873 (0.686 sec)
INFO:tensorflow:global_step/sec: 167.63
INFO:tensorflow:step = 13701, loss = 574.076 (0.597 sec)
INFO:tensorflow:global_step/sec: 159.496
INFO:tensorflow:step = 13801, loss = 688.941 (0.627 sec)
INFO:tensorflow:global_step/sec: 118.465
INFO:tensorflow:step = 

...

INFO:tensorflow:step = 717401, loss = 427.101 (0.531 sec)
INFO:tensorflow:global_step/sec: 231.964
INFO:tensorflow:step = 717501, loss = 558.065 (0.431 sec)
INFO:tensorflow:global_step/sec: 230.399
INFO:tensorflow:step = 717601, loss = 351.688 (0.434 sec)
INFO:tensorflow:global_step/sec: 230.807
INFO:tensorflow:step = 717701, loss = 365.309 (0.433 sec)
INFO:tensorflow:global_step/sec: 226.873
INFO:tensorflow:step = 717801, loss = 368.273 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.934
INFO:tensorflow:step = 717901, loss = 567.609 (0.439 sec)
INFO:tensorflow:global_step/sec: 209.854
INFO:tensorflow:step = 718001, loss = 467.848 (0.476 sec)
INFO:tensorflow:global_step/sec: 231.515
INFO:tensorflow:step = 718101, loss = 546.875 (0.432 sec)
INFO:tensorflow:global_step/sec: 226.524
INFO:tensorflow:step = 718201, loss = 401.285 (0.441 sec)
INFO:tensorflow:global_step/sec: 223.338
INFO:tensorflow:step = 718301, loss = 451.507 (0.448 sec)
INFO:tensorflow:global_step/sec: 227.957
INFO:tenso

INFO:tensorflow:step = 725701, loss = 546.218 (0.433 sec)
INFO:tensorflow:global_step/sec: 227.177
INFO:tensorflow:step = 725801, loss = 299.682 (0.440 sec)
INFO:tensorflow:global_step/sec: 228.645
INFO:tensorflow:step = 725901, loss = 366.733 (0.437 sec)
INFO:tensorflow:global_step/sec: 221.892
INFO:tensorflow:step = 726001, loss = 442.1 (0.451 sec)
INFO:tensorflow:global_step/sec: 212.893
INFO:tensorflow:step = 726101, loss = 437.412 (0.470 sec)
INFO:tensorflow:global_step/sec: 160.042
INFO:tensorflow:step = 726201, loss = 421.348 (0.625 sec)
INFO:tensorflow:global_step/sec: 212.485
INFO:tensorflow:step = 726301, loss = 488.806 (0.471 sec)
INFO:tensorflow:global_step/sec: 218.679
INFO:tensorflow:step = 726401, loss = 333.081 (0.457 sec)
INFO:tensorflow:global_step/sec: 225.685
INFO:tensorflow:step = 726501, loss = 427.974 (0.443 sec)
INFO:tensorflow:global_step/sec: 228.293
INFO:tensorflow:step = 726601, loss = 583.968 (0.438 sec)
INFO:tensorflow:global_step/sec: 229.275
INFO:tensorf

INFO:tensorflow:step = 734001, loss = 476.019 (0.434 sec)
INFO:tensorflow:global_step/sec: 228.013
INFO:tensorflow:step = 734101, loss = 456.802 (0.439 sec)
INFO:tensorflow:global_step/sec: 206.898
INFO:tensorflow:step = 734201, loss = 548.541 (0.483 sec)
INFO:tensorflow:global_step/sec: 228.009
INFO:tensorflow:step = 734301, loss = 406.631 (0.439 sec)
INFO:tensorflow:global_step/sec: 228.937
INFO:tensorflow:step = 734401, loss = 545.583 (0.437 sec)
INFO:tensorflow:global_step/sec: 232.448
INFO:tensorflow:step = 734501, loss = 465.701 (0.431 sec)
INFO:tensorflow:global_step/sec: 213.603
INFO:tensorflow:step = 734601, loss = 581.976 (0.468 sec)
INFO:tensorflow:global_step/sec: 231.857
INFO:tensorflow:step = 734701, loss = 471.816 (0.432 sec)
INFO:tensorflow:global_step/sec: 230.97
INFO:tensorflow:step = 734801, loss = 391.182 (0.433 sec)
INFO:tensorflow:global_step/sec: 227.095
INFO:tensorflow:step = 734901, loss = 383.002 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.051
INFO:tensor

INFO:tensorflow:step = 742301, loss = 319.866 (0.434 sec)
INFO:tensorflow:global_step/sec: 230.593
INFO:tensorflow:step = 742401, loss = 447.291 (0.434 sec)
INFO:tensorflow:global_step/sec: 232.235
INFO:tensorflow:step = 742501, loss = 586.66 (0.430 sec)
INFO:tensorflow:global_step/sec: 229.065
INFO:tensorflow:step = 742601, loss = 340.045 (0.437 sec)
INFO:tensorflow:global_step/sec: 233.178
INFO:tensorflow:step = 742701, loss = 333.818 (0.429 sec)
INFO:tensorflow:global_step/sec: 236.792
INFO:tensorflow:step = 742801, loss = 466.283 (0.422 sec)
INFO:tensorflow:global_step/sec: 233.939
INFO:tensorflow:step = 742901, loss = 546.68 (0.428 sec)
INFO:tensorflow:global_step/sec: 233.933
INFO:tensorflow:step = 743001, loss = 471.379 (0.427 sec)
INFO:tensorflow:global_step/sec: 218.904
INFO:tensorflow:step = 743101, loss = 400.77 (0.457 sec)
INFO:tensorflow:global_step/sec: 228.004
INFO:tensorflow:step = 743201, loss = 492.753 (0.438 sec)
INFO:tensorflow:global_step/sec: 231.895
INFO:tensorfl

INFO:tensorflow:step = 750601, loss = 435.319 (0.431 sec)
INFO:tensorflow:global_step/sec: 230.307
INFO:tensorflow:step = 750701, loss = 441.337 (0.434 sec)
INFO:tensorflow:global_step/sec: 223.066
INFO:tensorflow:step = 750801, loss = 483.458 (0.448 sec)
INFO:tensorflow:global_step/sec: 219.544
INFO:tensorflow:step = 750901, loss = 428.381 (0.456 sec)
INFO:tensorflow:global_step/sec: 227.636
INFO:tensorflow:step = 751001, loss = 386.853 (0.439 sec)
INFO:tensorflow:global_step/sec: 229.116
INFO:tensorflow:step = 751101, loss = 362.201 (0.437 sec)
INFO:tensorflow:global_step/sec: 217.96
INFO:tensorflow:step = 751201, loss = 461.671 (0.458 sec)
INFO:tensorflow:global_step/sec: 231.146
INFO:tensorflow:step = 751301, loss = 383.902 (0.433 sec)
INFO:tensorflow:global_step/sec: 232.262
INFO:tensorflow:step = 751401, loss = 514.961 (0.431 sec)
INFO:tensorflow:global_step/sec: 232.781
INFO:tensorflow:step = 751501, loss = 387.889 (0.430 sec)
INFO:tensorflow:Saving checkpoints for 751590 into .